# Faker for testing databases
Learn to use the `faker` module to allow testing database designs.

In [12]:
from faker import Faker

In [13]:
faker = Faker()

In [ ]:
print(faker.credit_card_full())

In [ ]:
faker.date_of_birth()

In [ ]:
faker.phone_number()

In [ ]:
print(faker.paragraph())

In [ ]:
faker.license_plate()

In [ ]:
faker.street_address()

In [ ]:
faker.coordinate()

In [ ]:
faker.words(4)

## Learn about Cursors
Cursors are the usual way of issuing database queries and processing their results.

In [1]:
creds = {'user': 'root', 'password': 'simple', 'host':'127.0.0.1'}

In [2]:
list(creds)

['user', 'password', 'host']

In [3]:
import pymysql

# establish a database connection
conn = pymysql.connect(
    host=creds['host'], 
    user=creds['user'], 
    passwd=creds['password'],
    autocommit=True,
)

In [4]:
cursor = conn.cursor(
    cursor=pymysql.cursors.DictCursor)

In [5]:
cursor.execute("CREATE SCHEMA IF NOT EXISTS university")

1

In [ ]:
s = '''
Yesterday, 
all my troubles seemed so far away.
Now it seems as though they're here to stay.
Oh, how I long for yesterday.
'''

In [6]:
cursor.execute('USE university')

0

In [7]:
cursor.execute("""
CREATE TABLE person(
person_id int not NULL,
first_name varchar(30) NOT NULL,
last_name varchar(30) NOT NULL,
date_of_birth date NOT NULL,
primary key(person_id)
)
"""
)

0

In [ ]:
cursor.execute("DROP TABLE person")

In [8]:
cursor.execute(
    """
    INSERT INTO person 
    VALUES
    (2, "Jane", "Doe", "2002-02-02"),
    (3, "John", "Smith", "2003-03-01"),
    (4, "John", "Wick", "1979-12-02")
""")

3

In [18]:
cursor.execute("""
    SELECT * FROM person
""")

4

In [19]:
for rec in cursor:
    print(rec)

{'person_id': 2, 'first_name': 'Jane', 'last_name': 'Doe', 'date_of_birth': datetime.date(2002, 2, 2)}
{'person_id': 3, 'first_name': 'John', 'last_name': 'Smith', 'date_of_birth': datetime.date(2003, 3, 1)}
{'person_id': 4, 'first_name': 'John', 'last_name': 'Wick', 'date_of_birth': datetime.date(1979, 12, 2)}
{'person_id': 5, 'first_name': 'Gregory', 'last_name': 'Morris', 'date_of_birth': datetime.date(1954, 9, 4)}


In [20]:
cursor.fetchall()

[]

In [14]:
# insert 
cursor.execute(
    """
    INSERT INTO person 
    (person_id, first_name, last_name, date_of_birth) VALUES
    (%s, %s, %s, %s)
""", (5, faker.first_name(), faker.last_name(), faker.date_of_birth()))

1

In [23]:
from tqdm import tqdm

In [25]:
for i in tqdm(range(1000, 1200)):
    cursor.execute("""
    INSERT INTO 
    person (person_id, first_name, last_name, date_of_birth) 
    VALUES (%s, %s, %s, %s)
    """, (i, faker.first_name(), faker.last_name(), faker.date_of_birth()))

100%|█████████████████████████████████████████| 200/200 [00:01<00:00, 164.26it/s]


In [26]:
cursor.execute("""
SELECT * FROM person
""")

204

In [28]:
cursor.fetchall()

[]

In [ ]:
cursor.execute("""
DROP TABLE dimitri_test.fake_person
""")

In [ ]:
import datetime

In [ ]:
cursor.execute("""USE dimitri_test""")

In [ ]:
cursor.execute("""SELECT * FROM fake_person""")

In [ ]:
cursor.fetchone()

In [ ]:
cursor.fetchall()

In [ ]:
cursor.execute("""SELECT * FROM dimitri_test.fake_person""")
for rec in cursor:
    print(rec)

In [ ]:
import datetime

In [ ]:
faker.date_between(datetime.date(2018, 2, 3), 'today')

In [ ]:
cursor.execute("""
drop table fake_death
""")

In [ ]:
cursor.execute("""
CREATE TABLE dimitri_test.fake_death(
    person_id int not null,
    date_of_death date NOT NULL,
    primary key(person_id), 
    foreign key (person_id) REFERENCES dimitri_test.fake_person (person_id))
""")

In [ ]:
cursor.execute("""
CREATE TABLE hotel_reserviation(
    
    hotel varchar(20) not null
    room  int not null,
    reservation_date date,
    person_id int not null,
    
    unique index (person_id, reservation_date),    
    primary key (hotel, room, reservation_date),
    foreign key (person_id) references fake_person(person_id)
""")

In [ ]:
cursor.execute("""
CREATE TABLE bank_account (
    
    bank_id  int not null, 
    account int not null,
    
    primary key(bank_id, account)

""")

cursor.execute("""
CREATE TABLE bank_account_owner (
    
    bank_id  int not null, 
    account int not null,
    person_id int not null,
    
    primary key(bank_id, account, person_id),
    foreign key (person_id) references fake_person(person_id),
    foreign key (bank_id, account) references fake_person(bank_id, account),

""")

In [ ]:
cursor.execute("""
SELECT * FROM fake_person
""")
cursor.fetchone()

In [ ]:
cursor.execute("""INSERT into fake_death (date_of_death) values ('2020-10-09')""")

In [ ]:
cursor.execute("""INSERT into fake_death (person_id, date_of_death) values (1000, '2020-09-09')""")

In [ ]:
persons = cursor.execute("""SELECT person_id, date_of_birth FROM dimitri_test.fake_person""")
for rec in cursor.fetchall():
    cursor.execute("""
    INSERT INTO dimitri_test.fake_death (person_id, date_of_death) VALUES (%s, %s)
    """, (rec['person_id'], faker.date_between(rec['date_of_birth'], rec['date_of_birth'] + datetime.timedelta(days=40000))))
    

In [ ]:
cursor.execute("""
SELECT first_name, floor(DATEDIFF(date_of_death, date_of_birth)/365.25) as died_at
FROM dimitri_test.fake_person NATURAL JOIN dimitri_test.fake_death""")

for rec in cursor:
    print(rec)

In [ ]:
cursor.execute("""
DROP TABLE dimitri_test.fake_death
""")

## Terminology

Translation from relational terminology into database programming

* Tuple -> Row
* Attribute -> Field/column
* Attribute value -> cell
* Relation -> Table
* Domain -> data type